In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
#install transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00


In [10]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [11]:
summarizer=pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [12]:
URL="https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
r=requests.get(URL)
r

<Response [200]>

In [13]:
soup= BeautifulSoup(r.text,"html.parser")
results=soup.find_all(["h1","p"])


In [16]:
text=[result.text for result in results]
article=" ".join(text)
article

'Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Listen Share I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babi

In [17]:
article=article.replace(".",".<eos>")
article=article.replace("!","!<eos>")
article=article.replace("?","?<eos>")
sentences=article.split("<eos>")

In [18]:
max_chunk=500
current_chunk=0
chunks=[]
for sentence in sentences:
  if len(chunks)==current_chunk+1:
    if len(chunks[current_chunk])+len(sentence.split(' ')) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk+=1
      chunks.append(sentence.split(" "))
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))

0


In [19]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id]=" ".join(chunks[chunk_id])

In [20]:
res= summarizer(chunks, max_length=120,min_length=30, do_sample=False)
res

[{'summary_text': ' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . The Bayesian approach is called analytic regularization for supervised learning .'},
 {'summary_text': ' We will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification, as well . We will focus on rigorously presenting the mathematics behind these claims . In the next section, we will use Bayes’ Rule to derive our L2 regularized estimator .'},
 {'summary_text': ' Using Bayes’ Rule, we can show that the mean and mode of the posterior distribution of w is the solution for LASSO regression when we invoke a Gaussian prior distribution on w . We’ll now examine a similar case with a Laplace prior . Here is the corresponding derivation for Lasso: (Note that in the last step, we set

In [21]:
text= " ".join([summ["summary_text"] for summ in res])

' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . The Bayesian approach is called analytic regularization for supervised learning .  We will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification, as well . We will focus on rigorously presenting the mathematics behind these claims . In the next section, we will use Bayes’ Rule to derive our L2 regularized estimator .  Using Bayes’ Rule, we can show that the mean and mode of the posterior distribution of w is the solution for LASSO regression when we invoke a Gaussian prior distribution on w . We’ll now examine a similar case with a Laplace prior . Here is the corresponding derivation for Lasso: (Note that in the last step, we set p = 1, q = 1)  A huge thank you to CODECOGS for their online 

In [23]:
with open("blogsummary.txt","w") as f:
  f.write(text)


TypeError: ignored